In [1]:
! pip install torchcontrib

In [2]:
import argparse
import json
import os
import sys
import warnings
from importlib import import_module
from pathlib import Path
from shutil import copy
from typing import Dict, List, Union
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
# from torch.utils.tensorboard import SummaryWriter
from torchcontrib.optim import SWA

from data_utils import (Dataset_ASVspoof2019_train,
                        Dataset_ASVspoof2019_devNeval, genSpoof_list)
from evaluation import calculate_tDCF_EER
from utilsa import create_optimizer, seed_worker, set_seed, str_to_bool

warnings.filterwarnings("ignore", category=FutureWarning)

seed = 123

torch.cuda.empty_cache()

***
__File Operations__

In [3]:
! wget "https://www.dropbox.com/s/36yqmymkva2bwdi/spcup_2022_training_part1.zip?dl=1" -c -O 'spcup_2022_training_part1.zip'
! wget "https://www.dropbox.com/s/wsmlthhri29fb79/spcup_2022_unseen.zip?dl=1" -c -O 'spcup_2022_unseen.zip'

In [ ]:
!unzip "./spcup_2022_training_part1.zip" -d "./spcup_2022_training/"
!unzip "./spcup_2022_unseen.zip" -d "./spcup_2022_unseen/"

In [5]:
!rm "./spcup_2022_training_part1.zip"
!rm "./spcup_2022_unseen.zip"

In [6]:
# create combined label df
# combine two label sets
df1 = pd.read_csv('./spcup_2022_training/spcup_2022_training_part1/labels.csv')
df2 = pd.read_csv('./spcup_2022_unseen/spcup_2022_unseen/labels.csv')
df3 = pd.concat([df1, df2]).sample(frac=1)

df3.to_csv('./final_labels.csv', index=False)
!rm './spcup_2022_unseen/spcup_2022_unseen/labels.csv'

In [7]:
# copy .wav files from unseen to all
!cp -a "./spcup_2022_unseen/spcup_2022_unseen/". "./spcup_2022_training/spcup_2022_training_part1/"

***

In [8]:
def get_model(model_config: Dict, device: torch.device):
    """Define DNN model architecture"""
    module = import_module("{}".format(model_config["architecture"]))
    _model = getattr(module, "Model")
    model = _model(model_config).to(device)
    nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
    print("no. model params:{}".format(nb_params))

    return model

In [9]:
def get_loader(
        database_path: str,
        label_path: str,
        config: dict) -> List[torch.utils.data.DataLoader]:
    """Make PyTorch DataLoaders for train / developement / evaluation"""
    # == dropping part ==
    # track = config["track"]
    # prefix_2019 = "ASVspoof2019.{}".format(track)

    # trn_database_path = database_path / "ASVspoof2019_{}_train/".format(track)
    # dev_database_path = database_path / "ASVspoof2019_{}_dev/".format(track)
    # eval_database_path = database_path / "ASVspoof2019_{}_eval/".format(track)

    # trn_list_path = (database_path /
    #                  "ASVspoof2019_{}_cm_protocols/{}.cm.train.trn.txt".format(
    #                      track, prefix_2019))
    # dev_trial_path = (database_path /
    #                   "ASVspoof2019_{}_cm_protocols/{}.cm.dev.trl.txt".format(
    #                       track, prefix_2019))
    # eval_trial_path = (
    #     database_path /
    #     "ASVspoof2019_{}_cm_protocols/{}.cm.eval.trl.txt".format(
    #         track, prefix_2019))

    # d_label_trn, file_train = genSpoof_list(dir_meta=trn_list_path,
    #                                         is_train=True,
    #                                         is_eval=False)
    # print("no. training files:", len(file_train))
    # =====================

    label_df = pd.read_csv(label_path)
    X, y = label_df['track'].values, label_df['algorithm'].values
    # stratified split dataset into train-validation
    # set param as config["split_ratio"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)


    train_set = Dataset_ASVspoof2019_train(list_IDs=X_train,
                                           labels=y_train,
                                           base_dir=database_path)
    gen = torch.Generator()
    gen.manual_seed(seed)
    trn_loader = DataLoader(train_set,
                            batch_size=config["batch_size"],
                            shuffle=True,
                            drop_last=True,
                            pin_memory=True,
                            worker_init_fn=seed_worker,
                            generator=gen)

    # # == test dataset not yet given ==

    # _, file_dev = genSpoof_list(dir_meta=dev_trial_path,
    #                             is_train=False,
    #                             is_eval=False)
    # print("no. validation files:", len(file_dev))

    # dev_set = Dataset_ASVspoof2019_devNeval(list_IDs=file_dev,
    #                                         base_dir=dev_database_path)
    # dev_loader = DataLoader(dev_set,
    #                         batch_size=config["batch_size"],
    #                         shuffle=False,
    #                         drop_last=False,
    #                         pin_memory=True)
    # =================================

    # == validation dataset {updated} == 

    eval_set = Dataset_ASVspoof2019_train(list_IDs=X_test,
                                           labels=y_test,
                                           base_dir=database_path)
    eval_loader = DataLoader(eval_set,
                             batch_size=config["batch_size"],
                             shuffle=False,
                             drop_last=False,
                             pin_memory=True,
                             worker_init_fn=seed_worker,
                             generator=gen)

    return trn_loader, eval_loader

In [10]:
def produce_evaluation_file(
    data_loader: DataLoader,
    model,
    device: torch.device,
    save_path: str,
    trial_path: str) -> None:
    """Perform evaluation and save the score to a file"""
    model.eval()
    with open(trial_path, "r") as f_trl:
        trial_lines = f_trl.readlines()
    fname_list = []
    score_list = []
    for batch_x, utt_id in data_loader:
        batch_x = batch_x.to(device)
        with torch.no_grad():
            _, batch_out = model(batch_x)
            batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
        # add outputs
        fname_list.extend(utt_id)
        score_list.extend(batch_score.tolist())

    assert len(trial_lines) == len(fname_list) == len(score_list)
    with open(save_path, "w") as fh:
        for fn, sco, trl in zip(fname_list, score_list, trial_lines):
            _, utt_id, _, src, key = trl.strip().split(' ')
            assert fn == utt_id
            fh.write("{} {} {} {}\n".format(utt_id, src, key, sco))
    print("Scores saved to {}".format(save_path))

In [11]:
def train_epoch(
    trn_loader: DataLoader,
    model,
    optim: Union[torch.optim.SGD, torch.optim.Adam],
    device: torch.device,
    scheduler: torch.optim.lr_scheduler,
    config: argparse.Namespace):
    """Train the model for one epoch"""
    
    running_loss = 0
    num_total = 0.0
    train_acc, correct_train, target_count = 0, 0, 0
    ii = 0
    model.train()
    scaler = GradScaler()

    # set objective (Loss) functions
    weight = torch.FloatTensor([0.1, 0.9]).to(device)
    # criterion = nn.CrossEntropyLoss(weight=weight)
    criterion = nn.CrossEntropyLoss()
    for batch_x, batch_y in trn_loader:
        batch_size = batch_x.size(0)
        num_total += batch_size
        ii += 1
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_y = batch_y.view(-1).to(device)
        _, batch_out = model(batch_x)#, Freq_aug=str_to_bool(config["freq_aug"]))
        batch_loss = criterion(batch_out, batch_y)
        running_loss += batch_loss.item() * batch_size
        optim.zero_grad()
        scaler.scale(batch_loss).backward()
        # batch_loss.backward()
        scaler.step(optim)
        scaler.update()
        # optim.step()

        if config["optim_config"]["scheduler"] in ["cosine", "keras_decay"]:
            scheduler.step()
        elif scheduler is None:
            pass
        else:
            raise ValueError("scheduler error, got:{}".format(scheduler))
        
        # accuracy
        _, predicted = torch.max(batch_out.data, 1)
        target_count += batch_y.size(0)
        correct_train += (batch_y == predicted).sum().item()
        train_acc = (100 * correct_train) / target_count

    running_loss /= num_total
    return running_loss, train_acc

In [12]:
def eval_epoch(
    trn_loader: DataLoader,
    model,
    device: torch.device,
    config: argparse.Namespace):
    """Train the model for one epoch"""
    running_loss = 0
    num_total = 0.0
    val_acc, correct_train, target_count = 0, 0, 0
    ii = 0
    model.eval()

    # set objective (Loss) functions
    weight = torch.FloatTensor([0.1, 0.9]).to(device)
    # criterion = nn.CrossEntropyLoss(weight=weight)
    criterion = nn.CrossEntropyLoss()
    for batch_x, batch_y in trn_loader:
        batch_size = batch_x.size(0)
        num_total += batch_size
        ii += 1
        batch_x = batch_x.to(device)
        # batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_y = batch_y.view(-1).to(device)
        _, batch_out = model(batch_x) #model(batch_x, Freq_aug=str_to_bool(config["freq_aug"]))
        batch_loss = criterion(batch_out, batch_y)
        running_loss += batch_loss.item() * batch_size
        
        # accuracy
        _, predicted = torch.max(batch_out.data, 1)
        target_count += batch_y.size(0)
        correct_train += (batch_y == predicted).sum().item()
        val_acc = (100 * correct_train) / target_count
        
    running_loss /= num_total
    return running_loss, val_acc

In [13]:
config = {
    
    "database_path": "./spcup_2022_training/spcup_2022_training_part1/",
    "label_path": "./final_labels.csv",
    "asv_score_path": "ASVspoof2019_LA_asv_scores/ASVspoof2019.LA.asv.eval.gi.trl.scores.txt",
    "model_path": "./models/weights/AASIST.pth",
    "batch_size": 8,  # cloud not use higher value, because of CUDA memory overflow
    "num_epochs": 100,
    "loss": "CCE",
    "track": "LA",
    "eval_all_best": "True",
    "eval_output": "eval_scores_using_best_dev_model.txt",
    "cudnn_deterministic_toggle": "True",
    "cudnn_benchmark_toggle": "False",
    "model_config": {
        "architecture": "aasist",
        "nb_samp": 64600,
        "first_conv": 128,
        "filts": [70, [1, 32], [32, 32], [32, 64], [64, 64]],
        "gat_dims": [64, 32],
        "pool_ratios": [0.5, 0.7, 0.5, 0.5],
        "temperatures": [2.0, 2.0, 100.0, 100.0]
    },
    "optim_config": {
        "optimizer": "adam", 
        "amsgrad": "False",
        "base_lr": 0.0001,
        "lr_min": 0.000005,
        "betas": [0.9, 0.999],
        "weight_decay": 0.0001,
        "scheduler": "cosine"
    }
}

In [14]:
args = argparse.Namespace()
args.config = config
args.seed = seed
args.output_dir = './tmp/out'
args.comment = False

In [15]:
def main(args):

    # define database related paths
    output_dir = Path(args.output_dir)
    database_path = Path(config["database_path"])
    label_path = Path(config['label_path'])

    # set device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device: {}".format(device))
    # if device == "cpu":
    #     raise ValueError("GPU not detected!")

    # define model architecture
    model_config = args.config["model_config"]
    model = get_model(model_config, device)

    # define dataloaders
    trn_loader, eval_loader = get_loader(database_path, label_path, config)

    # get optimizer and scheduler
    optim_config = config["optim_config"]
    optim_config["epochs"] = config["num_epochs"]
    optim_config["steps_per_epoch"] = len(trn_loader)
    optimizer, scheduler = create_optimizer(model.parameters(), optim_config)
    optimizer_swa = SWA(optimizer)

    # Training
    for epoch in range(config["num_epochs"]):
        print("Start training epoch{:03d}".format(epoch))
        training_loss, training_acc = train_epoch(trn_loader, model, optimizer, device,
                                   scheduler, config)
        eval_loss, eval_acc = eval_epoch(eval_loader, model, device, config)
        print(f'[{epoch}] Training Loss : {training_loss} / Training Accuracy : {training_acc} | Eval Loss : {eval_loss} / Eval Accuracy : {eval_acc}')

In [16]:
main(args)